In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,28367
2,Huelva,Confirmados PDIA 14 días,1166
3,Huelva,Tasa PDIA 14 días,"227,21515287331684"
4,Huelva,Confirmados PDIA 7 días,542
5,Huelva,Total Confirmados,28542
6,Huelva,Curados,23974
7,Huelva,Fallecidos,363


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  28367.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7349.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1199 personas en los últimos 7 días 

Un positivo PCR cada 537 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,28367.0,542.0,1166.0,513170.0,105.618021,227.215153,165.0
Condado-Campiña,8890.0,282.0,603.0,156231.0,180.501949,385.966934,87.0
Huelva-Costa,16556.0,209.0,439.0,289548.0,72.181469,151.615622,68.0
Huelva (capital),7349.0,120.0,268.0,143837.0,83.427769,186.322017,38.0
Almonte,1489.0,73.0,182.0,24507.0,297.874077,742.644959,21.0
Sierra de Huelva-Andévalo Central,2531.0,48.0,109.0,67391.0,71.226128,161.742666,17.0
Moguer,1088.0,38.0,87.0,21867.0,173.777839,397.859789,16.0
Bollullos Par del Condado,799.0,36.0,66.0,14387.0,250.225898,458.747480,13.0
Bonares,252.0,16.0,28.0,6060.0,264.026403,462.046205,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cumbres Mayores,49.0,7.0,24.0,1749.0,400.228702,1372.212693,1.0
Cumbres de San Bartolomé,10.0,2.0,5.0,376.0,531.914894,1329.787234,0.0
Lucena del Puerto,224.0,33.0,37.0,3261.0,1011.959522,1134.621282,8.0
Cortelazor,7.0,2.0,3.0,299.0,668.896321,1003.344482,2.0
Galaroza,27.0,3.0,11.0,1382.0,217.076700,795.947902,0.0
Almonte,1489.0,73.0,182.0,24507.0,297.874077,742.644959,21.0
Rociana del Condado,503.0,11.0,47.0,7939.0,138.556493,592.014108,6.0
Beas,194.0,17.0,22.0,4341.0,391.614835,506.795669,2.0
Bonares,252.0,16.0,28.0,6060.0,264.026403,462.046205,8.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Rociana del Condado,503.0,11.0,47.0,7939.0,138.556493,592.014108,6.0,0.234043
Palma del Condado (La),1030.0,4.0,17.0,10801.0,37.033608,157.392834,1.0,0.235294
Aljaraque,992.0,4.0,16.0,21474.0,18.627177,74.508708,1.0,0.250000
Almonaster la Real,65.0,2.0,8.0,1788.0,111.856823,447.427293,2.0,0.250000
Galaroza,27.0,3.0,11.0,1382.0,217.076700,795.947902,0.0,0.272727
Cumbres Mayores,49.0,7.0,24.0,1749.0,400.228702,1372.212693,1.0,0.291667
Niebla,193.0,2.0,6.0,4116.0,48.590865,145.772595,1.0,0.333333
Aroche,90.0,1.0,3.0,3054.0,32.743942,98.231827,1.0,0.333333
Gibraleón,582.0,9.0,25.0,12737.0,70.660281,196.278559,1.0,0.360000
